In [17]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
from torch.utils.data import DataLoader
from t5_dataset import T5Dataset
import torch

The model is hosted on Hugging Face: https://huggingface.co/krkv/energy-t5-large

It is downloaded by `transformers` package on the first run (around 3GB) and cached locally for the next runs.

In [18]:
t5_model_name = "krkv/energy-t5-large"
device = "cpu"
model = T5ForConditionalGeneration.from_pretrained(t5_model_name)
model = model.to(device)
model.eval()
tokenizer = T5Tokenizer.from_pretrained(t5_model_name)

/home/rodion/TrustAI/llm-evaluation/t5/venv-t5-eval/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Reading the gold parse dataset:

In [23]:
df = pd.read_csv("../gold_parse_energy.csv")
input_sentences = df.iloc[:,0].values
input_parses = df.iloc[:,1].values
input_size = len(input_sentences)

Input sentences are appended with the instruction prompt and wrapped with a T5 dataset class:

In [20]:
instruction = "Convert the question into an SQL parse: "

sentences_with_instruction = [
    instruction + sentence
    for sentence in input_sentences
]

empty_targets = [""] * len(sentences_with_instruction)

prediction_dict = {"source": sentences_with_instruction, "predict": empty_targets}

prediction_df = pd.DataFrame(prediction_dict)

prediction_data_set = T5Dataset(dataframe=prediction_df,
                                tokenizer=tokenizer,
                                source_len=128,
                                target_len=128,
                                source_text="source",
                                target_text="predict")

prediction_data_loader = DataLoader(prediction_data_set,
                                    batch_size=16,
                                    shuffle=False,
                                    num_workers=0)

This loop handles loading of data in batches and generating of parses by calling `model.generate`:

In [21]:
generated_parses = []

with torch.no_grad():
    for data in prediction_data_loader:
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        generated_ids = model.generate(
            input_ids=ids,
            attention_mask=mask,
            max_length=150,
            early_stopping=True
        )
        
        generation_text = [
            tokenizer.decode(tokens, skip_special_tokens=True, clean_up_tokenization_spaces=True).lower()
            for tokens in generated_ids
        ]

        generated_parses.extend(generation_text)

/home/rodion/TrustAI/llm-evaluation/t5/venv-t5-eval/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Make a calculation of accuracy as percent of correct parses over the entire dataset:

In [24]:
correct_parses = 0
correct_percent = 0

for i in range(input_size):
    print("Expected: " + input_parses[i], "- Generated: " + generated_parses[i])
    if input_parses[i] == generated_parses[i]:
        correct_parses += 1

if correct_parses > 0:
    correct_percent = round((correct_parses / input_size) * 100, 2)
    
print("\n==========\n")
print("Calculated model accuracy: " + str(correct_percent) + "%")

Expected: function [e] - Generated: function [e]
Expected: self [e] - Generated: self [e]
Expected: self and function [e] - Generated: previousfilter and explain features [e]
Expected: self [e] - Generated: self and function [e]
Expected: function [e] - Generated: previousfilter and explain features [e]
Expected: data [e] - Generated: data [e]
Expected: model [e] - Generated: model [e]
Expected: data and model [e] - Generated: data and model [e]
Expected: model and data [e] - Generated: model and data [e]
Expected: features [e] - Generated: define [e]


Calculated model accuracy: 60.0%
